## Домашнее задание № 7 по курсу "MLOps"
#### Обучение сложных моделей машинного обучения с помощью распределенных инструментов
##### Автор: Кравченя Павел

##### Цели работы:
Тренировка обучения модель градиентного бустинга в распределенном режиме с помощью Spark.

##### Постановка задачи:

1. Скачать [датасет](https://www.kaggle.com/sharthz23/sna-hackathon-2019-collaboration/download). В нем содержатся табличные данные показов ленты социальной сети ``ok.ru`` за 1,5 месяца. Для анализа следует использовать данные, расположенные в директории ``sna-hackathon-2019/train``.

2. Подготовить признаки на основе датасета.

3. Обучить модель градиентного бустинга на Spark.

4. Оценить качество модели.

Работа выполнялась с использованием Docker-образа системы ``almond.sh`` с версией ``Scala 2.12`` и ``Spark 3.1.0``.

##### Сборка Docker-образа
Поскольку для работы градиентного бустинга требовалась библиотека OpenMP, которая отсутствовала в Docker-образе ``almond.sh``, было принято решение собрать на его основе новый образ, доустановив библиотеку. Последовательность действий для сборки образа и запуска проекта:

``git clone https://github.com/kpdphys/MLOps.git``

``cd MLOps/Task5``

``docker build -t openmp-almondsh:latest ./docker``

``docker run -p 8888:8888 --rm -v $(pwd):/home/jovyan/work --name openmp-almondsh openmp-almondsh:latest``

После успешных сборки образа и запуска контейнера веб-интерфейс ``jupyter notebook (Almondsh)`` будет доступен на порту ``8888``.

##### Выполнение работы

`Замечание` Поскольку работа с градиентным бустингом на Spark уже осуществлялась автором в пятом домашнем задании, а с нейронной сетью -- в третьем, то, **по согласованию с руководителем курса**, было решено сделать следующее. К уже имеющемуся анализу датасета, который осуществлялся с использованием градиентного бустинга, добавить анализ с применением простой линейной модели (логистической регрессии) и сравнить результаты. Поэтому, первая часть работы будет повторять содержание пятого домашнего задания.

Для построения модели и выполнения ее интерпретации воспользуемся библиотекой Microsoft [SynapseML](https://microsoft.github.io/SynapseML). Установим ее и другие необходимые библиотеки для выполнения вычислений и визуализации.

In [ ]:
import $ivy.`org.apache.spark::spark-sql:3.1.0`
import $ivy.`com.microsoft.azure::synapseml:0.9.5`
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`

In [ ]:
import org.apache.log4j.{Level, Logger}
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{Imputer, OneHotEncoder, StringIndexer, VectorAssembler, StandardScaler}
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import com.microsoft.azure.synapse.ml.lightgbm._
import org.apache.spark.ml.functions.vector_to_array

Установим необходимый уровень логирования сообщений и создадим Spark-сессию.

In [ ]:
import org.apache.log4j.{Level, Logger}
import org.apache.spark.sql._
Logger.getLogger("org").setLevel(Level.OFF)

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .config("spark.executor.memory", "32g")
    .config("spark.driver.memory", "32g")
    .getOrCreate()
}

Загрузим датасет и выведем схему данных.

In [ ]:
  val data = spark.read.parquet("sna-hackathon-2019/train")
  data.printSchema()

Так как датасет содержит большое количество признаков, выберем из них те, значения которых вероятнее всего будут влиять на отток пользователей. Вместе с названиями отобранных признаков сохраним их тип (``категориальные``('c') или ``количественные`` ('q')), поскольку от него будет зависеть преобработка данных.

In [ ]:
val features_types = Map(
    "instanceId_objectType" -> 'c',
    "audit_clientType" -> 'c',
    "audit_resourceType" -> 'c',
    "membership_status" -> 'c',
    "user_gender" -> 'c',
    "user_status" -> 'c',
    "user_ID_country" -> 'c',
    "metadata_ownerType" -> 'c',
    "metadata_platform" -> 'c',
    "userOwnerCounters_USER_PROFILE_VIEW" -> 'c',
    "userOwnerCounters_USER_SEND_MESSAGE" -> 'c',
    "userOwnerCounters_USER_INTERNAL_LIKE" -> 'c',
    "userOwnerCounters_USER_STATUS_COMMENT_CREATE" -> 'c',
    "userOwnerCounters_USER_FORUM_MESSAGE_CREATE" -> 'c',
    "userOwnerCounters_PHOTO_COMMENT_CREATE" -> 'c',
    "userOwnerCounters_COMMENT_INTERNAL_LIKE" -> 'c',
    "userOwnerCounters_MOVIE_COMMENT_CREATE" -> 'c',
    "userOwnerCounters_USER_PHOTO_ALBUM_COMMENT_CREATE" -> 'c',

    "auditweights_userAge" -> 'q',
    "metadata_numCompanions" -> 'q',
    "userOwnerCounters_CREATE_LIKE" -> 'q',
    "auditweights_ageMs" -> 'q',
    "auditweights_ctr_gender" -> 'q',
    "auditweights_ctr_high" -> 'q',
    "auditweights_ctr_negative" -> 'q',
    "auditweights_dailyRecency" -> 'q',
    "auditweights_feedStats" -> 'q',
    "auditweights_friendCommentFeeds" -> 'q',
    "auditweights_friendCommenters" -> 'q',
    "auditweights_friendLikes" -> 'q'
  )

Определим пользователей как ушедших (``churn``), если они не совершали никаких действий с системой **в течение двух недель**. Для этого определим текущее на момент формирования датасета время как время самого последнего события в системе.

In [ ]:
import org.apache.spark.sql.functions._
val curr_timestamp = data.select("audit_timestamp").agg(max("audit_timestamp")).take(1)(0).getLong(0)

Всех пользователей, для которых последняя запись о любом их взаимодействии с системой обнаруживается раньше, чем за две недели до вычисленной даты, разметим как ушедших ``(is_churn)``.

In [ ]:
import spark.implicits._
val wait_time_days: Int = 14
val labeled_data = data.select("instanceId_userId", "audit_timestamp")
    .groupBy("instanceId_userId")
    .agg(max("audit_timestamp")
      .as("last_att_timestamp"))
    .withColumn("curr_timestamp", lit(curr_timestamp))
    .withColumn("delta_timestamp", $"curr_timestamp" - $"last_att_timestamp")
    .withColumn("days_absent", $"delta_timestamp" / (24 * 3600 * 1000))
    .withColumn("is_churn", when($"days_absent" > wait_time_days, 1.0).otherwise(0.0))

labeled_data.show()

Добавим метку к каждому пользователю для каждой записи в датасете и отфильтруем признаки, оставив только ранее выбранные.

In [ ]:
val clean_labeled_data = data
    .join(labeled_data, data("instanceId_userId") === labeled_data("instanceId_userId"), "left")
    .select((features_types.toArray.map(x => col(x._1)) :+ col("is_churn")): _*)

Определим количества записей в датасете отдельно для ушедших и оставшихся пользователей, а также их отношение.

In [ ]:
val churn_count = clean_labeled_data.filter($"is_churn" === 1.0).count
val non_churn_count = clean_labeled_data.filter($"is_churn" === 0.0).count
val churn_ratio = non_churn_count.toDouble / churn_count
println(f"churn_count = $churn_count, non_churn_count = $non_churn_count, ratio = $churn_ratio%.3f")

Видно, что данных, имеющих отношение к ушедшим пользователям, меньше в 4 раза, чем к оставшимся. Выполним процедуру ``undersampling`` для балансировки датасета перед обучением.

In [ ]:
def underample_df(df: DataFrame): DataFrame = {
    val churn_df = df.filter($"is_churn" === 1.0)
    val non_churn_df = df.filter($"is_churn" === 0.0)
    val sample_ratio = churn_df.count().toDouble / df.count().toDouble
    val non_churn_sampled_Df = non_churn_df.sample(false, sample_ratio)
    churn_df.union(non_churn_sampled_Df)
  }

Разобьем стратифицированно (т.е. с соблюдением баланса классов) датасет на обучающую и тестовую выборки в пропорции ``70:30``. Перемешаем полученные выборки для более равномерного обучения.

In [ ]:
val splitted_data = clean_labeled_data.randomSplit(Array(0.7, 0.3), seed = 1234L)
val train_dataset = underample_df(splitted_data(0)).orderBy(rand())
val test_dataset = splitted_data(1).orderBy(rand())

Построим модель на основе градиентного бустинга над деревьями решений. Реализацию алгоритма бустинга возьмем из библиотеки ``LightGBM``, которая является составной частью ``SynapseML``. При создании классификатора укажем столбец результирующего датасета, в который будут записаны коэффициенты Шепли, необходимые для интерпретации обученной модели.

In [ ]:
val clsfr_gbm = new LightGBMClassifier()
    .setFeaturesCol("features")
    .setLabelCol("is_churn")
    .setObjective("binary")
    .setFeaturesShapCol("shapValues")

Создадим конвейер данных, в котором перед обучением классификатора категориальные данные будут кодироваться посредством ``LabelEncoding``, а в количественных данных будут заполнены пропуски.

In [ ]:
val pipeline_gbm = new Pipeline()
    .setStages(Array(
      new StringIndexer()
        .setHandleInvalid("keep")
        .setInputCols(features_types.filter(x => x._2 == 'c').toArray.map(x => x._1))
        .setOutputCols(features_types.filter(x => x._2 == 'c').toArray.map(x => x._1 + "_ind")),
      new Imputer()
        .setInputCols(features_types.filter(x => x._2 == 'q').toArray.map(x => x._1))
        .setOutputCols(features_types.filter(x => x._2 == 'q').toArray.map(x => x._1 + "_imp")),
      new VectorAssembler()
        .setInputCols(features_types.filter(x => x._2 == 'q').toArray.map(x => x._1 + "_imp") ++ 
                      features_types.filter(x => x._2 == 'c').toArray.map(x => x._1 + "_ind"))
        .setOutputCol("features"),
      clsfr_gbm
))

Обучим модель.

In [ ]:
val model_gbm = pipeline_gbm.fit(train_dataset)

Определим предсказания обученной модели на тренировочной и тестовой части датасета.

In [ ]:
val train_predictions_gbm = model_gbm.transform(train_dataset)
val test_predictions_gbm  = model_gbm.transform(test_dataset)

Для оценки качества модели воспользуемся объектом ``BinaryClassificationEvaluator``, который вычислит значения ``AUC ROC`` для данных.

In [ ]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

val evaluator = new BinaryClassificationEvaluator()
    .setLabelCol("is_churn")
    .setMetricName("areaUnderROC")

Выведем рассчитанные значения ``AUC ROC`` для тренировочной и тестовой частей датасета.

In [ ]:
val areaUnderROC_train_gbm = evaluator.evaluate(train_predictions_gbm)
val areaUnderROC_test_gbm  = evaluator.evaluate(test_predictions_gbm)

Получившееся значение ~67% как на тренировочной, так и на тестой частях датасета является небольшим, но его можно признать удовлетворительным для Baseline-модели. Рассчитаем также другие метрики классификации на тестовой выборке.

In [ ]:
import com.microsoft.azure.synapse.ml.train.ComputeModelStatistics
val stats = new ComputeModelStatistics()
    .setLabelCol("is_churn")
    .setScoresCol("probability_1")
    .setScoredLabelsCol("prediction")
    .setEvaluationMetric("classification")

In [ ]:
import org.apache.spark.ml.functions.vector_to_array
val metrics_gbm = stats.transform(test_predictions_gbm.
                                  withColumn("probability_1", 
                                             vector_to_array($"probability").getItem(1)))
metrics_gbm.select("accuracy", "precision", "recall", "AUC").show(false)

Получили Acc = ~64%, Pr = ~64%, Rc = ~82%. Аналогично вышерассмотренному случаю, метрики можно считать удовлетворительными для Baseline-модели.

`Замечание` При разных запусках расчета метрик для градиентного бустинга результаты могут получаться существенно разными. Выше приведены наиболее высокие значения, которых удалось добиться. Причины такого поведения пока не выяснены.

Сформируем небольшую выборку из тестовой части датасета, состоящую из объектов, на которых модель выдала максимальную вероятность ухода. Они соответствуют пользователям, которые, по мнению модели, уйдут в самое ближайшее время. Именно для этих объектов попробуем объяснить влияние на вероятность ухода клиента различных признаков, на которых обучалась модель.

In [ ]:
val explain_instances_gbm = test_predictions_gbm
    .filter($"prediction" === 1.0)
    .withColumn("probability_1", vector_to_array($"probability").getItem(1))
    .orderBy($"probability_1".desc)
    .limit(40)

Сформируем функцию, которая получает значения коэффициентов Шепли для рассматриваемых объектов, усредняет их и визуализирует несколько самых больших по модулю значений.

In [ ]:
def plot_shap_gbm(shap_data: Dataset[_], top_obj_count: Int): Unit = {
    import org.apache.spark.ml.linalg.DenseVector
    import org.apache.spark.ml.stat.Summarizer
    
    def get_features_from_dataframe(data: Dataset[_]): Array[String] = {
        val meta = data
            .schema("features")
            .metadata
            .getMetadata("ml_attr")
            .getMetadata("attrs")

        val meta_numeric = if(meta.contains("numeric")) {
            meta
            .getMetadataArray("numeric")
            .map(x => (x.getLong("idx"), x.getString("name")))
        } else { Array[(Long, String)]() }
    
        val meta_nominal = if(meta.contains("nominal")) {
            meta
            .getMetadataArray("nominal")
            .map(x => (x.getLong("idx"), x.getString("name")))
        } else { Array[(Long, String)]() }
    
        val meta_binary = if(meta.contains("binary")) {
            meta
            .getMetadataArray("binary")
            .map(x => (x.getLong("idx"), x.getString("name")))
        } else { Array[(Long, String)]() }
    
        (meta_numeric ++ meta_nominal ++ meta_binary).sortBy(_._1).map(_._2)
    }
    
    val shaps = shap_data
        .select("shapValues")
        .groupBy()
        .agg(Summarizer.mean($"shapValues").alias("means"))
        .map { case Row(shapValues_1: DenseVector) => shapValues_1.toArray } collect
    
    val shaps_with_features = shaps(0)
        .drop(1)
        .zip(get_features_from_dataframe(shap_data))
        .sortBy(_._1)
    
    val filtered_shaps_with_features = shaps_with_features.slice(0, top_obj_count) ++
        shaps_with_features.slice(shaps_with_features.size - top_obj_count, 
                                  shaps_with_features.size)
    
    filtered_shaps_with_features.foreach(x => println(x._2))
    
    val data = Seq(Bar(
        filtered_shaps_with_features.map(_._1).toSeq,
        filtered_shaps_with_features.map(_._2).toSeq,
        orientation = Orientation.Horizontal
    ))
    
    plot(data)
}

Выполним визуализацию усредненных коэффициентов Шепли для клиентов, вероятность которых уйти, согласно обученной модели, наибольшая.

In [ ]:
plot_shap_gbm(explain_instances_gbm, 6)

Из гистограммы можно наблюдать, что самое большое влияние на вероятность ухода оказали следующие признаки:
* ``user_status`` (статус пользователя)
* ``auditweights_ctr_negative`` (рейтинг кликов пользователя)
* ``user_gender`` (пол пользователя)
* ``membership_status`` (информация о членстве пользователя в группе, где опубликован контент)

Таким образом, статус пользователя непосредственно влияет на вероятность ухода. Возможно, данный признак непосредственно рассчитывается с учетом ушедших клиентов, и тогда является утечкой данных при анализе :) Сказать заранее нельзя, поскольку алгоритм его расчета неизвестен.

Рейтинг кликов и членство пользователя в группах показывают заинтересованность пользователя в контенте социальной сети. Следует предложить бизнес-аналитикам продумать способы привлечения внимания новых клиентов к предоставляемому им материалу, что, возможно, снизит риск оттока пользователей.

С учетом того, что пол клиента имеет большое влияние на вероятность оттока клиентов, следует порекомендовать бизнес-аналитикам проанализировать материал на ориентированность только на мужскую или женскую аудиторию. Также, следует исключить возможные варианты сексизма в контенте пользователей.

Теперь попробуем проанализировать признаки, оказывающие большое влияние на уход пользователя, с помощью логистической регрессии. Дальшнейшие шаги будут схожи с вышеописанными для градиентного бустинга.

In [ ]:
import org.apache.spark.ml.classification.LogisticRegression
val clsfr_lr = new LogisticRegression()
    .setFeaturesCol("features")
    .setLabelCol("is_churn")

Для линейных моделей количественные признаки необходимо отмасштабировать, а категориальные -- закодировать с помощью `one-hot`.

In [ ]:
val pipeline_lr = new Pipeline()
    .setStages(Array(
      new StringIndexer()
        .setHandleInvalid("keep")
        .setInputCols(features_types.filter(x => x._2 == 'c').toArray.map(x => x._1))
        .setOutputCols(features_types.filter(x => x._2 == 'c').toArray.map(x => x._1 + "_ind")),
      
      new OneHotEncoder()
        .setInputCols(features_types.filter(x => x._2 == 'c').toArray.map(x => x._1 + "_ind"))
        .setOutputCols(features_types.filter(x => x._2 == 'c').toArray.map(x => x._1 + "_ohe")),
        
      new Imputer()
        .setInputCols(features_types.filter(x => x._2 == 'q').toArray.map(x => x._1))
        .setOutputCols(features_types.filter(x => x._2 == 'q').toArray.map(x => x._1 + "_imp")),
        
      new VectorAssembler()
        .setInputCols(features_types.filter(x => x._2 == 'q').toArray.map(x => x._1 + "_imp"))
        .setOutputCol("q_imp"),  
        
      new StandardScaler()
        .setInputCol("q_imp")
        .setOutputCol("q_norm"),

      new VectorAssembler()
        .setInputCols(Array("q_norm") ++ features_types.filter(x => x._2 == 'c').toArray.map(x => x._1 + "_ohe"))
        .setOutputCol("features"),
    
      clsfr_lr
))

In [ ]:
val model_lr = pipeline_lr.fit(train_dataset)

In [ ]:
val train_predictions_lr = model_lr.transform(train_dataset)
val test_predictions_lr = model_lr.transform(train_dataset)

Рассчитаем значения AUC ROC для линейной модели.

In [ ]:
val areaUnderROC_train_lr = evaluator.evaluate(train_predictions_lr)
val areaUnderROC_test_lr  = evaluator.evaluate(test_predictions_lr)

И train, и test показали AUC ROC ~65%. Это примерно на 2% меньше результатов, достигнутых с использованием градиентного бустинга. Как говорилось ранее, для baseline-модели это -- удовлетворительное значение.

In [ ]:
val metrics_lr = stats.transform(test_predictions_lr
                                 .withColumn("probability_1", 
                                             vector_to_array($"probability").getItem(1)))
metrics_lr.select("accuracy", "precision", "recall", "AUC").show(false)

Получили долю верных ответов (accuracy) ~62%, точность (precision) ~63%, полноту (recall) ~76%. Результаты схожи с аналогичными данными, полученными с использованием градиентного бустинга.

Вновь отберем 40 пользователей, вероятность ухода которых, согласно модели, максимальна.

In [ ]:
val explain_instances_lr = test_predictions_lr
    .filter($"prediction" === 1.0)
    .withColumn("probability_1", vector_to_array($"probability").getItem(1))
    .orderBy($"probability_1".desc)
    .limit(40)

При обучении модели градиентного бустинга с использованием библиотеки `SynapseML` коээфициенты Шепли считались библиотекой во время обучения. При использовании модели логистической регрессии эти коэффициенты необходимо посчитать отдельно.

In [ ]:
import org.apache.spark.ml.classification.LogisticRegressionModel
import com.microsoft.azure.synapse.ml.explainers.VectorSHAP

val shap_lr = new VectorSHAP()
  .setInputCol("features")
  .setOutputCol("shapValues_Vector")
  .setNumSamples(5000)
  .setModel(model_lr.stages.last.asInstanceOf[LogisticRegressionModel])
  .setTargetCol("probability")
  .setTargetClasses(Array(1))
  .setBackgroundData(test_predictions_lr.orderBy(rand()).limit(10))

In [ ]:
val shap_df_lr = shap_lr.transform(explain_instances_lr)

Преобразуем датасет путем извлечения из полученного вектора нужных коэффициентов.

In [ ]:
val shap_values_df_lr = shap_df_lr.withColumn("shapValues", $"shapValues_Vector".getItem(0))

In [ ]:
def plot_shap_lr(shap_data: Dataset[_], 
                 features_types: Map[String, Char], 
                 top_obj_count: Int): Unit = {
    import org.apache.spark.ml.linalg.DenseVector
    import org.apache.spark.ml.stat.Summarizer
    
    def get_features_from_dataframe(data: Dataset[_]): Array[String] = {
        val meta = data
            .schema("features")
            .metadata
            .getMetadata("ml_attr")
            .getMetadata("attrs")

        val meta_numeric = if(meta.contains("numeric")) {
            val q_features = features_types
                .filter(x => x._2 == 'q')
                .toArray
                .map(x => x._1)
                .zipWithIndex
                .map(x => ("q_norm_" + x._2 -> x._1))
                .toMap
            meta
            .getMetadataArray("numeric")
            .map(x => (x.getLong("idx"), q_features(x.getString("name"))))
        } else { Array[(Long, String)]() }
    
        val meta_nominal = if(meta.contains("nominal")) {
            meta
            .getMetadataArray("nominal")
            .map(x => (x.getLong("idx"), x.getString("name")))
        } else { Array[(Long, String)]() }
    
        val meta_binary = if(meta.contains("binary")) {
            meta
            .getMetadataArray("binary")
            .map(x => (x.getLong("idx"), x.getString("name")))
        } else { Array[(Long, String)]() }
    
        (meta_numeric ++ meta_nominal ++ meta_binary).sortBy(_._1).map(_._2)
    }
    
    val shaps = shap_data
        .select("shapValues")
        .groupBy()
        .agg(Summarizer.mean($"shapValues").alias("means"))
        .map { case Row(shapValues_1: DenseVector) => shapValues_1.toArray } collect
    
    val shaps_with_features = shaps(0)
        .drop(1)
        .zip(get_features_from_dataframe(shap_data))
        .sortBy(_._1)
    
    val filtered_shaps_with_features = shaps_with_features.slice(0, top_obj_count) ++
        shaps_with_features.slice(shaps_with_features.size - top_obj_count, 
                                  shaps_with_features.size)
    
    filtered_shaps_with_features.foreach(x => println(x._2))
    
    val data = Seq(Bar(
        filtered_shaps_with_features.map(_._1).toSeq,
        filtered_shaps_with_features.map(_._2).toSeq,
        orientation = Orientation.Horizontal
    ))
    
    plot(data)
}

In [ ]:
plot_shap_lr(shap_values_df_lr, features_types, 6)

Согласно линейной модели, наибольшее влияние на уход пользователя оказывают:
1. Признак `auditweights_dailyRecency`, который, судя по названию, непосредственно связан с ежедневной активностью пользователя.
2. Признак `auditweights_ageMs`, который, возможно, связан с возрастом пользователя (сложно понять из названия).

Интересной особенностью является тот факт, что признаки, отобранные логистической регрессией и градиентным бустингом, в большинстве своем не совпадают. Только признак, коррелирующий с возрастом пользователя, фигурирует в результатах обеих моделей.

Также, была выявлена следующая особенность. При различных запусках обучения модели и валидации результатов `lightgbm` выдает метрики, которые могут значительно отличаться. При этом, метрики, полученные с помощью логистической регрессии, практически не изменяются. Объяснения этому факту пока не найдено...

#### Выводы:
В процессе выполнения работы был проанализирован датасет показов ленты социальной сети ok.ru за 1,5 месяца с целью определения признаков, оказывающих наибольшее влияние на отток пользователей. Был выбран критерий оттока, с учетом которого датасет был размечен. Над размеченным датасетом решалась задача бинарной классификации (``churn``/ ``not churn``) с помощью классификаторов на основе градиентного бустинга и логистической регрессии. Проверка качества обученной модели показала, что метрики классификации можно считать удовлетворительными для Baseline-модели. Степень влияния различных признаков на вероятность оттока была проанализирована с помощью коэффициентов Шепли, которые были рассчитаны с использованием библиотеки ``SynapseML`` и визуализированы. По результатам анализа были сформулированы способы удержания наиболее серьезно настроенных уйти пользователей.

Результаты и метрики, демонстрируемые линейной моделью, оказались схожими с результатами градиентного бустинга. Однако, признаки, которые две модели считают наиболее влияющими на готовность пользователей уйти, различаются.

В результате работы были получены навыки работы с библиотекой ``SynapseML`` под ``Scala``, изучен способ интерпретации моделей машинного обучения с помощью коэффициентов Шепли.